# ROBOCLAW TUTORIAL<hr>
Written by: Quentin Demory<br>
August 2020 @ SSRS<br>
MIT license<hr>
## introduction
ROBOCLAWS are smart motor controllers, equipped with onboard EEPROM, sensors (current, voltage, temperature...), securities (max voltage, max current...), a LOGICS side (Raspberry Pi style male connectors) to monitor speed, position, acceleration and decceleration of motors and a 'POWER' side where the energy is delivered, to effectively move the motors, via encoder regulated PWM or brute PWM orders (such as proceed forward at half speed).<br>
Before continuing on, let us look at what the difference between what I labelled as 'encoder regulated' and 'brute' PWM.
+ _encoder regulated_: 
```python
>>> SpeedDistanceM1(address, speed, distance, buffer)
```
SpeedDistanceM1: Choose your speed and distance on channel 1 (there are 2 channels on every ROBOCLAWS, M1 & M2)<br>
address: set up in ION motion studio always starts at 128, and increments unp to 8 ROBOCLAWS in serie (129, 130, ...)
speed: defined by QPPS (Quadrature Pulses Per Second), the motor's maximum speed. That comes as a raw value that differ for each motor. Say I defined (through tuning in ION motion studio) my QPPS to be 50 000. I would therefore have 0 be my minimum speed and 50 000 be maximum. This defines our 0 - 100% and as such if I want to run at 50% speed capacity I should enter 25 000.<br>
distance: also defined in ION motion studio (IMS), you can choose your min and max (raw encoder) values. say we start at 0 and I decide that my motor will travel a distance of 1 meter. Looking at the encoder value displayed in IMS, say that it is 100 000.<br>
buffer: this can only be set to '0' or '1'. '1' means that you want a command such as SpeedDistanceM1 to take effect immediately. '0' means that you want to back it up so that your command will take effect when the current command has cleared. **This buffer/cascade system only works on commands placed on the same channel of the same ROBOCLAW**
here it is:
```python
>>> SpeedDistanceM1(128, 25000, 100000, 1)
>>> SpeedDistanceM1(128, -25000, 100000, 0)
```
Here. I asked the motor 1, channel 1 to proceed 1 meter forward at 50% of its max speed and once that was done, to proceed 1 meter backwards at half speed.
+ _brute_:
```python
>>> ForwardM1(address, val)
```
ForwardM1: proceed forward on channel 1
address: same as above
val: any value between 0 and 127, respectively representing 0% and 100% of you output PWM signal. 50 % would sit at about 64.
```python
>>> ForwardM1(128, 64)
```
Here, I am again proceeding forward on motor 1 channel 1 at 50% of the speed but I have to manually stop when the motion has reached the meter that I aimed to travel to and then order it to go backwards`
```python
>>> ForwardM1(128, 0) #stop
>>> BackwardM1(128, 64) 
```
then wait until it reached the meter again, the point of origin of the distance covered and 
```python
>>> ForwardM1(128, 0) #stop
```
A little more involved...<br>
There is a lot of fun to be had with the ROBOCLAWS and as you can see from the very quick overview of two methods there are various ways to get motion.<br>
**NB**: the above commands are not valid. I called them methods as they are defined within the Roboclaw class, found in the roboclaw_3 module and thus an object must be created to dispatch the methods to. if you are not familiar with classes, they are a way to contain and assign functions (methods) to a variable (object). you will soon see it being assigned in the line 
```rc = Roboclaw()```. Now ```rc``` will have the properties of the Roboclaw class and calling the above methods legitimately will look like 
```
>>> rc.SpeedDistanceM1(128, 25000, 100000, 1)
```

## set up

To get successfully set up, make sure that your roboclaw_3.py module is in the same folder you are working from. If you do not have the module on your machine, it can be retrieved in the downloads of BasicMicro, the ROBOCLAW makers. **you will notice** that I import from the roboclaw library instead of roboclaw_3 and it is because I deleted the roboclaw library (python 2) that comes in the download and relabelled roboclaw_3 to roboclaw. The download from BasicMicro comes loaded with many examples but they are all (as of now) python 2 examples. download them as they are fun to test with but know that you will have to reformat a bit so it is compatible with python 3 (add parenthesis to print statements, remove the commas at the end of lines and change the ```print format(" ")``` to ```print(" ".format())``` depending on which python version you are on. ```print repr(" ")``` also comes up and is reformatted to ```print(repr(" "))```... you will find more descrepancies but they are usually confined to print statements).<br>Let's set our Claw up:

In [7]:
from roboclaw import Roboclaw

In [8]:
#Open serial port
#Linux comport name - RPi
rc = Roboclaw("/dev/ttyACM0",115200)
#Windows comport name
#rc = Roboclaw("COM8",115200)
rc.Open()

0

In [9]:
rc.SpeedAccelDeccelPositionM1(129, 400000, 50000, 300000, 10000, 1)

AttributeError: 'Roboclaw' object has no attribute '_port'